In [1]:
import numpy as np
import scipy as sp
import scipy.signal as sig
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers, regularizers, callbacks
from data_wrangling.datamanager import DataLoader
import pickle
import sys

In [7]:
dl = DataLoader()
data = dl.get_fcx2(['s1'])

In [8]:
X1 = data['s1']['data']
mn = X1.mean(0).reshape((1, -1))
st = X1.std (0).reshape((1, -1))
X1 = (X1 - mn) / st
X1.shape

(1303952, 73)

In [11]:
fs = 1000

idxs_train_test_split = int(len(X1) * 0.7)

p_in  = 100
p_out = 10

step  = 20 

idxs_train = np.arange(0, idxs_train_test_split - p_out - p_in, step)

X = np.stack([
    X1[idx : idx + p_in]
    for idx in idxs_train
])

Y = np.stack([
    X1[idx + p_in : idx + p_in + p_out]
    for idx in idxs_train
])

X.shape, Y.shape

((45633, 100, 73), (45633, 10, 73))

In [44]:
model = keras.Sequential([
    layers.Input(X.shape[1:]),
    layers.Bidirectional(
        layers.LSTM(
            64, 
            return_sequences = True
        )
    ),
    layers.Lambda(lambda inputs: inputs[:, -p_out:, :]),
    layers.Dense(128),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.Dense(Y.shape[-1])
])

model.compile(loss = 'mse', optimizer = 'adam')
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_10 (Bidirectio (None, 100, 128)          70656     
_________________________________________________________________
lambda_8 (Lambda)            (None, 10, 128)           0         
_________________________________________________________________
dense_7 (Dense)              (None, 10, 128)           16512     
_________________________________________________________________
batch_normalization_1 (Batch (None, 10, 128)           512       
_________________________________________________________________
re_lu_1 (ReLU)               (None, 10, 128)           0         
_________________________________________________________________
dense_8 (Dense)              (None, 10, 73)            9417      
Total params: 97,097
Trainable params: 96,841
Non-trainable params: 256
________________________________________________

In [45]:
hist = model.fit(
    X_train,
    Y_train,
    epochs = 50,
    batch_size = 2048,
    validation_split = 0.2,
    callbacks = [callbacks.EarlyStopping(min_delta = 1e-3, patience = 2)]
)

Epoch 1/50
18/18 [==============================] - 4s 131ms/step - loss: 1.1972 - val_loss: 0.7250
Epoch 2/50
18/18 [==============================] - 2s 102ms/step - loss: 0.4966 - val_loss: 0.5628
Epoch 3/50
18/18 [==============================] - 2s 101ms/step - loss: 0.3716 - val_loss: 0.5064
Epoch 4/50
18/18 [==============================] - 2s 101ms/step - loss: 0.3086 - val_loss: 0.4699
Epoch 5/50
18/18 [==============================] - 2s 114ms/step - loss: 0.2734 - val_loss: 0.4505
Epoch 6/50
18/18 [==============================] - 2s 103ms/step - loss: 0.2472 - val_loss: 0.4375
Epoch 7/50
18/18 [==============================] - 2s 103ms/step - loss: 0.2268 - val_loss: 0.4175
Epoch 8/50
18/18 [==============================] - 2s 105ms/step - loss: 0.2128 - val_loss: 0.4097
Epoch 9/50
18/18 [==============================] - 2s 103ms/step - loss: 0.1994 - val_loss: 0.4006
Epoch 10/50
18/18 [==============================] - 2s 103ms/step - loss: 0.1885 - val_loss: 0.3905